<a href="https://colab.research.google.com/github/AnudeepReddy-Katta/RNNs-LSTM-Attention/blob/main/Part_2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd

import random
import math
import time

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
!pip install spacy --upgrade

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.6)


In [ ]:
%%bash
python -m spacy download en
# python -m spacy download de

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-06-24 04:06:18.127155: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
# spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
# def tokenize_de(text):
#     """
#     Tokenizes German text from a string into a list of strings (tokens) and reverses it
#     """
#     return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
# # importing the "tarfile" module
# import tarfile
  
# # open file
# file = tarfile.open('/content/drive/MyDrive/s7/Question_Answer_Dataset_v1.2.tar.gz')
  
# # extracting file
# file.extractall('/content/drive/MyDrive/s7')
  
# file.close()

In [ ]:
s08_data = pd.read_csv("/content/drive/MyDrive/s7/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt", sep='\t',encoding='ISO-8859-1')
s09_data = pd.read_csv("/content/drive/MyDrive/s7/Question_Answer_Dataset_v1.2/S09/question_answer_pairs.txt", sep='\t',encoding='ISO-8859-1')
s10_data = pd.read_csv("/content/drive/MyDrive/s7/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt", sep='\t', quoting=3,encoding='ISO-8859-1')

In [ ]:
print(s08_data.shape)
print(s09_data.shape)
print(s10_data.shape)

(1715, 6)
(825, 6)
(1458, 6)


In [ ]:
print(s08_data.columns)
print(s09_data.columns)
print(s10_data.columns)

Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')
Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')
Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')


In [ ]:
s08_data = s08_data.drop(columns=['ArticleTitle','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'])
s09_data = s09_data.drop(columns=['ArticleTitle','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'])
s10_data = s10_data.drop(columns=['ArticleTitle','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'])

In [ ]:
QA_data = pd.concat([s08_data,s09_data,s10_data], axis =0)
print(QA_data.head())
print(QA_data.shape)

                                            Question Answer
0  Was Abraham Lincoln the sixteenth President of...    yes
1  Was Abraham Lincoln the sixteenth President of...   Yes.
2  Did Lincoln sign the National Banking Act of 1...    yes
3  Did Lincoln sign the National Banking Act of 1...   Yes.
4                   Did his mother die of pneumonia?     no
(3998, 2)


In [ ]:
QA_data.Question = np.array(QA_data.Question).astype(str)
QA_data.Answer = np.array(QA_data.Answer).astype(str)

In [ ]:
Question = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

Answer = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
from sklearn.model_selection import train_test_split

train, test_data = train_test_split(QA_data, test_size=0.3)
train = train.reset_index().drop(columns=['index'])
test_data = test_data.reset_index().drop(columns=['index'])
train_data, valid_data = train_test_split(train, test_size=0.25)
train_data = train_data.reset_index().drop(columns=['index'])
valid_data = valid_data.reset_index().drop(columns=['index'])

In [ ]:
fields = [('Question', Question),('Answer',Answer)]

In [ ]:
example = [torchtext.legacy.data.Example.fromlist([train_data.Question[i],train_data.Answer[i]], fields) for i in range(train_data.shape[0])]

In [ ]:
trainDataset = torchtext.legacy.data.Dataset(example, fields)

In [ ]:
example = [torchtext.legacy.data.Example.fromlist([test_data.Question[i],test_data.Answer[i]], fields) for i in range(test_data.shape[0])]

In [ ]:
testDataset = torchtext.legacy.data.Dataset(example, fields)

In [ ]:
example = [torchtext.legacy.data.Example.fromlist([valid_data.Question[i],valid_data.Answer[i]], fields) for i in range(valid_data.shape[0])]

In [ ]:
validDataset = torchtext.legacy.data.Dataset(example, fields)

In [ ]:
print(f"Number of training examples: {len(trainDataset.examples)}")
print(f"Number of validation examples: {len(validDataset.examples)}")
print(f"Number of testing examples: {len(testDataset.examples)}")

Number of training examples: 2098
Number of validation examples: 700
Number of testing examples: 1200


In [ ]:
print(vars(trainDataset.examples[0]))

{'Question': ['was', 'arabic', 'a', 'major', 'vehicle', 'of', 'culture', 'in', 'europe', '?'], 'Answer': ['nan']}


In [ ]:
Question.build_vocab(trainDataset)
Answer.build_vocab(trainDataset)

In [ ]:
print(f"Unique tokens in Question vocabulary: {len(Question.vocab)}")
print(f"Unique tokens in Answer vocabulary: {len(Answer.vocab)}")

Unique tokens in Question vocabulary: 3474
Unique tokens in Answer vocabulary: 2501


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (trainDataset, validDataset, testDataset), 
    batch_size = BATCH_SIZE, sort=False, 
    device = device)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = len(Question.vocab)
OUTPUT_DIM = len(Answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3474, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(2501, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=2501, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,169,029 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
Answer_PAD_IDX = Answer.vocab.stoi[Answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = Answer_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Question
        trg = batch.Answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question
            trg = batch.Answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 3s
	Train Loss: 6.005 | Train PPL: 405.370
	 Val. Loss: 5.247 |  Val. PPL: 190.008
Epoch: 02 | Time: 0m 2s
	Train Loss: 5.133 | Train PPL: 169.483
	 Val. Loss: 5.368 |  Val. PPL: 214.338
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.996 | Train PPL: 147.811
	 Val. Loss: 5.426 |  Val. PPL: 227.291
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.912 | Train PPL: 135.851
	 Val. Loss: 5.470 |  Val. PPL: 237.541
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.838 | Train PPL: 126.190
	 Val. Loss: 5.530 |  Val. PPL: 252.092
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.760 | Train PPL: 116.755
	 Val. Loss: 5.595 |  Val. PPL: 269.046
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.699 | Train PPL: 109.882
	 Val. Loss: 5.645 |  Val. PPL: 282.888
Epoch: 08 | Time: 0m 2s
	Train Loss: 4.620 | Train PPL: 101.492
	 Val. Loss: 5.731 |  Val. PPL: 308.136
Epoch: 09 | Time: 0m 2s
	Train Loss: 4.594 | Train PPL:  98.854
	 Val. Loss: 5.747 |  Val. PPL: 313.281
Epoch: 10 | Time: 0m 2s
	Train Loss: 4.539 | Train PPL:  93.616


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.245 | Test PPL: 189.560 |
